Importing libraries

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

Creating spark Session

In [0]:
spark=SparkSession.builder.appName("Google_Play_Store").getOrCreate()

Reading csv file

In [0]:
google_df=spark.read.format("csv")\
                .option("header","True")\
                  .option("inferschema","True")\
                    .load("/FileStore/tables/googleplaystore.csv")

In [0]:
google_df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



Dropping Unnecessary Columns

In [0]:
google_df=google_df.drop("Size","Content Rating","Last Updated","Current Ver","Android Ver")

In [0]:
google_df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [0]:
google_df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



Casting DataTypes

In [0]:
google_df=google_df.withColumn("Rating",col("Rating").cast(DoubleType()))\
                  .withColumn("Reviews",col("Reviews").cast(IntegerType()))\
                    .withColumn("Installs",regexp_replace("Installs","[^0-9]",""))\
                      .withColumn("Installs",col("Installs").cast(IntegerType()))\
                        .withColumn("Price",regexp_replace("Price","[$]",""))\
                          .withColumn("Price",col("Price").cast(IntegerType()))

In [0]:
google_df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [0]:
google_df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



Top 10 app based on total reviews

In [0]:
top10_review_wise=google_df.groupBy("App")\
                            .agg(sum("Reviews").alias("total_reviews"))\
                                .orderBy("total_reviews",ascending=False)\
                                    .limit(10)

In [0]:
top10_review_wise.show(truncate=False)

+----------------------------------------+-------------+
|App                                     |total_reviews|
+----------------------------------------+-------------+
|Instagram                               |266241989    |
|WhatsApp Messenger                      |207348304    |
|Clash of Clans                          |179558781    |
|Messenger – Text and Video Chat for Free|169932272    |
|Subway Surfers                          |166331958    |
|Candy Crush Saga                        |156993136    |
|Facebook                                |156286514    |
|8 Ball Pool                             |99386198     |
|Clash Royale                            |92530298     |
|Snapchat                                |68045010     |
+----------------------------------------+-------------+



Top 10 installed Apps and distribution of types(paid/free)

In [0]:
top10_installed_app=google_df.groupBy("App","Type")\
                              .agg(sum("Installs").alias("total_installation"))\
                                .orderBy(desc("total_installation"))\
                                  .limit(10)
                                        

In [0]:
top10_installed_app.show()

+------------------+----+------------------+
|               App|Type|total_installation|
+------------------+----+------------------+
|    Subway Surfers|Free|        6000000000|
|         Instagram|Free|        4000000000|
|      Google Drive|Free|        4000000000|
|          Hangouts|Free|        4000000000|
|     Google Photos|Free|        4000000000|
|       Google News|Free|        4000000000|
|  Candy Crush Saga|Free|        3500000000|
|WhatsApp Messenger|Free|        3000000000|
|             Gmail|Free|        3000000000|
|      Temple Run 2|Free|        3000000000|
+------------------+----+------------------+



Category Wise Distribution of Installed Apps

In [0]:
category_wise_dist=google_df.groupBy("Category")\
                              .agg(sum("Installs").alias("total_installation"))\
                                .orderBy("total_installation",ascending=False)

In [0]:
category_wise_dist.show(10)

+------------------+------------------+
|          Category|total_installation|
+------------------+------------------+
|              GAME|       35086024415|
|     COMMUNICATION|       32647276251|
|      PRODUCTIVITY|       14176091369|
|            SOCIAL|       14069867902|
|             TOOLS|       11452771915|
|            FAMILY|       10258263505|
|       PHOTOGRAPHY|       10088247655|
|NEWS_AND_MAGAZINES|        7496317760|
|  TRAVEL_AND_LOCAL|        6868887146|
|     VIDEO_PLAYERS|        6222002720|
+------------------+------------------+
only showing top 10 rows



Genres Wise Distribution of Paid Installed Apps

In [0]:
genres_wise_apps=google_df.groupBy("Genres","Type")\
                            .agg(sum("Installs").alias("total_installation"))\
                                .filter(col("Type")=="Paid")\
                                    .orderBy(desc("total_installation"))

In [0]:
genres_wise_apps.show()

+--------------------+----+------------------+
|              Genres|Type|total_installation|
+--------------------+----+------------------+
|Arcade;Action & A...|Paid|          20110000|
|              Action|Paid|          17052250|
|     Personalization|Paid|           5258794|
|         Photography|Paid|           3978740|
|              Puzzle|Paid|           2190620|
|        Role Playing|Paid|           2092100|
|            Strategy|Paid|           1995500|
|               Tools|Paid|           1727441|
|              Arcade|Paid|           1437215|
|        Productivity|Paid|           1412055|
|       Communication|Paid|           1360050|
|           Adventure|Paid|           1278200|
|              Sports|Paid|           1243815|
|Education;Pretend...|Paid|           1188000|
|           Lifestyle|Paid|           1179110|
|  Puzzle;Brain Games|Paid|           1100000|
|             Medical|Paid|           1020033|
|Card;Action & Adv...|Paid|           1000000|
|            